In [1]:
# Setup scenario
from copy import deepcopy
import json
from pathlib import Path

from constants import DATA_ENTITIES_DIR, DATA_HAZARDS_DIR, DATA_TEMP_DIR, REQUIREMENTS_DIR
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from shapely.geometry import Point

from climada.engine import CostBenefit, Impact, ImpactCalc
from climada.engine.cost_benefit import risk_aai_agg, risk_rp_100, risk_rp_250
from climada.entity import DiscRates, Entity, Exposures, LitPop
from climada.entity.impact_funcs import ImpactFunc, ImpactFuncSet, ImpfTropCyclone
from climada.entity.impact_funcs.storm_europe import ImpfStormEurope
from climada.entity.impact_funcs.trop_cyclone import ImpfSetTropCyclone
from climada.entity.measures import Measure, MeasureSet
from climada.hazard import Hazard
from climada.util.api_client import Client

from costben.costben_handler import CostBenefitHandler
from entity.entity_handler import EntityHandler
from exposure.exposure_handler import ExposureHandler
from hazard.hazard_handler import HazardHandler
from impact.impact_handler import ImpactHandler

from base_handler import BaseHandler
from logger_config import LoggerConfig

logger = LoggerConfig(logger_types=["file"])

base_handler = BaseHandler()
costben_handler = CostBenefitHandler()
entity_handler = EntityHandler()
exposure_handler = ExposureHandler()
hazard_handler = HazardHandler()
impact_handler = ImpactHandler()


# Available Exposure data types in CLIMADA API for Egypt/Thailand: ['litpop']
# Available Hazard data types in CLIMADA API for Egypt/Thailand: ['river_flood', 'wildfire', 'earthquake', flood, 'tropical_cyclone']
# Available climate scenarios for hazard type river_flood/tropical_c in country Egypt/Thailand: ['rcp26', 'historical', 'rcp60', 'rcp85']
# Available time horizons for hazard type river_flood in country Egypt: ['2030_2050', '1980_2000', '2070_2090', '2010_2030', '2050_2070']

country_name = "Egypt"
exposure_type = "litpop"  # Available exposure types for Egypt/Thailand: ['litpop']
hazard_type = "river_flood"  # Available hazard types for Egypt/Thailand: ['river_flood', 'wildfire', 'earthquake', 'flood', 'tropical_cyclone']
scenario = "rcp26"  # Available scenarios for Egypt/Thailand: ['rcp26', 'historical', 'rcp60', 'rcp85']
time_horizon = "2030_2050"  # Available time horizons for Egypt/Thailand: ['2030_2050', '1980_2000', '2070_2090', '2010_2030', '2050_2070']
annual_growth = 1.01

client = Client()

# growth_rates = {
#     "Egypt": {
#         "crops": 0.04,
#         "livestock": 0.04,
#         "power_plants": 0.04,
#         "hotels": 0.04,
#         "hospitalised_people": 0.0129,
#         "students": 0.0129,
#         "diarrhea_patients": 0.0129,
#         "roads": 0.0129,
#     },
#     "Thailand": {
#         "tree_crops": 0.0294,
#         "grass_crops": 0.0294,
#         "wet_markets": 0.0294,
#         "grass_crops_farmers": -0.0022,
#         "tree_crops_farmers": -0.0022,
#         "buddhist_monks": -0.0022,
#         "diarrhea_patients": -0.0022,
#         "students": -0.0022,
#         "roads": -0.0022,
#     },
# }

In [ ]:
# Read data files
import h5py
import rasterio

def read_mat():
    with h5py.File(DATA_HAZARDS_DIR / "hazard_HW_EGY_rcp45.mat", 'r') as f:
        # Print all items in the root
        print("Items in the root:", list(f.keys()))

        # Access the 'hazard' group/dataset
        hazard = f['hazard']
        print("Items in 'hazard':", list(hazard.keys()))

        # If 'hazard' contains further groups or datasets, access them
        for item in hazard:
            print(f"Exploring {item}:")
            data = hazard[item]
            
            # Check if this is a dataset or a group
            if isinstance(data, h5py.Dataset):
                print(f"Dataset {item} found with shape {data.shape} and data type {data.dtype}")
            else:
                print(f"Group {item} contains: {list(data.keys())}")

            # If the item is stored by reference (common in MATLAB structures)
            if data.dtype == 'O':  # Object references
                # This will go through each reference and try to resolve it
                for ref in data:
                    referenced_object = f[data[ref][0]]  # Access by reference
                    print(f"Referenced object for {ref}: {referenced_object.shape}")

                    # Optionally, you can load the data into an array or similar
                    # print(np.array(referenced_object))

def read_tif():
    with rasterio.open(DATA_HAZARDS_DIR / "hazard_HW_EGY_rcp45.tif") as src:
        print(f'Number of bands: {src.count}')
        # Loop through each band
        # for i in range(1, src.count + 1):
        #     band = src.read(i)
        #     print(band)
        #     meta = src.tags(i)
        #     print(f'  Metadata for Band {i}: {meta}')



def explore_h5_file(file_path):
    """
    Reads and explores an HDF5 (.h5) file, printing its structure and basic data info.

    :param file_path: The path to the HDF5 file.
    :type file_path: str or Path
    """
    def explore_group(group, level=0):
        """
        Recursively explores a group in the HDF5 file, printing dataset information.

        :param group: The HDF5 group to explore.
        :type group: h5py.Group
        :param level: The current depth level in the group hierarchy (for indentation).
        :type level: int
        """
        indent = "  " * level  # To visualize hierarchy

        for key in group.keys():
            item = group[key]
            if isinstance(item, h5py.Group):
                print(f"{indent}Group: {key}")
                # Recursively explore this group
                explore_group(item, level + 1)
            elif isinstance(item, h5py.Dataset):
                print(f"{indent}Dataset: {key}")
                print(f"{indent}  Shape: {item.shape}")
                print(f"{indent}  Data type: {item.dtype}")
                # Print attributes if available
                if item.attrs:
                    print(f"{indent}  Attributes: {dict(item.attrs)}")
            else:
                print(f"{indent}Other: {key}")

    # Open the HDF5 file
    with h5py.File(file_path, 'r') as f:
        print(f"Exploring HDF5 file: {file_path}")
        # Explore the root group
        explore_group(f)

# Example usage
explore_h5_file(DATA_HAZARDS_DIR / "Thai_DR_RCP_45.h5")


In [2]:
# Example App
entity_present = entity_handler.get_entity_from_xlsx(
    DATA_ENTITIES_DIR / "entity_TODAY_THA_D_grass_crops.xlsx" # type: ignore
)
entity_present.check()
entity_present.exposures.ref_year = 2024

entity_future = entity_handler.get_future_entity(entity_present, 2050, 0.0294)
entity_future.check()

hazard_present = hazard_handler.get_hazard(
    hazard_type="drought", filepath=DATA_HAZARDS_DIR / "hazard_D_THA_historical.h5"
)
hazard_present.haz_type = "D"
hazard_present.check()

hazard_future = hazard_handler.get_hazard(
    hazard_type="drought", filepath=DATA_HAZARDS_DIR / "hazard_D_THA_rcp45.h5"
)
hazard_future.haz_type = "D"
hazard_future.check()

hazard_present.units = hazard_handler.get_hazard_intensity_units_from_entity(entity_present)
hazard_future.units = hazard_handler.get_hazard_intensity_units_from_entity(entity_future)

hazard_present.centroids.set_geometry_points()
hazard_future.centroids.set_geometry_points()

hazard_present.intensity_thres = hazard_handler.get_hazard_intensity_thres("D")
hazard_future.intensity_thres = hazard_handler.get_hazard_intensity_thres("D")

# Calculate impact
impact_present = ImpactCalc(
    entity_present.exposures, entity_present.impact_funcs, hazard_present
).impact(save_mat=True)
impact_future = ImpactCalc(
    entity_future.exposures, entity_future.impact_funcs, hazard_future
).impact(save_mat=True)


costben = costben_handler.calculate_cost_benefit(
    hazard_present, entity_present, hazard_future, entity_future, 2050
)
# costben_handler.plot_cost_benefit(costben)
# costben_handler.plot_waterfall(
#     costben, hazard_present, entity_present, hazard_future, entity_future
# )
# ax = costben.plot_waterfall(hazard_present, entity_present, hazard_future, entity_future)
# costben.plot_cost_benefit()

# exposure_handler.generate_exposure_geojson(entity_future.exposures, "Thailand")
# hazard_handler.generate_hazard_geojson(hazard_future, "Thailand")
# impact_handler.generate_impact_geojson(impact_future, "Thailand")


Measure      Cost (USD )    Benefit (USD )    Benefit/Cost
---------  -------------  ----------------  --------------
SWP          6.61653e+06                 0               0
CSA          1.28705e+07                 0               0
WSS          2.142e+07                   0               0
WDF          1.05e+08                    0               0

--------------------  -  ------
Total climate risk:   0  (USD )
Average annual risk:  0  (USD )
Residual risk:        0  (USD )
--------------------  -  ------
Net Present Values


In [3]:
entity_future.exposures.gdf

,category_id,latitude,longitude,value,value_unit,deductible,cover,impf_,region_id,centr_D
0,202,18.294191,103.706008,35160.720197,USD,205026,0,202,NaN,599
1,202,18.294191,103.714991,16834.905270,USD,205027,0,202,NaN,599
2,202,18.303174,103.661092,95840.912659,USD,205028,0,202,NaN,599
3,202,18.303174,103.670075,10773.933441,USD,205029,0,202,NaN,599
4,202,18.303174,103.714991,30829.250486,USD,205030,0,202,NaN,599
...,...,...,...,...,...,...,...,...,...,...
453243,202,17.377909,103.894654,61241.697464,USD,658269,0,202,NaN,835
453244,202,17.377909,103.903637,102724.971329,USD,658270,0,202,NaN,835
453245,202,17.386892,103.876688,97660.766819,USD,658271,0,202,NaN,835
453246,202,17.386892,103.885671,33040.200964,USD,658272,0,202,NaN,835


In [17]:
import numpy as np
from scipy.spatial import cKDTree

exp_lat = entity_future.exposures.gdf["latitude"].to_numpy(dtype=float)
exp_lon = entity_future.exposures.gdf["longitude"].to_numpy(dtype=float)
haz_lat = hazard_future.centroids.coord[:, 0].astype(float)
haz_lon = hazard_future.centroids.coord[:, 1].astype(float)

# ---- Quick bbox reject test ----
exp_bbox = (exp_lat.min(), exp_lat.max(), exp_lon.min(), exp_lon.max())
haz_bbox = (haz_lat.min(), haz_lat.max(), haz_lon.min(), haz_lon.max())
print("EXP bbox (lat_min,lat_max,lon_min,lon_max):", exp_bbox)
print("HAZ bbox (lat_min,lat_max,lon_min,lon_max):", haz_bbox)

# ---- Project to local-equirectangular (km) for fast KDTree ----
R = 6371.0088  # km
phi0 = np.deg2rad(np.mean(exp_lat))  # reference latitude

def to_xy_km(lat_deg, lon_deg):
    lat = np.deg2rad(lat_deg)
    lon = np.deg2rad(lon_deg)
    x = R * lon * np.cos(phi0)
    y = R * lat
    return np.column_stack([x, y])

haz_xy = to_xy_km(haz_lat, haz_lon)
exp_xy = to_xy_km(exp_lat, exp_lon)

# ---- KDTree NN distances from exposures to nearest hazard centroid ----
tree = cKDTree(haz_xy)
dist_km, _ = tree.query(exp_xy, k=1, workers=-1)  # vectorized; returns km due to our projection

# ---- Stats + thresholds ----
thr_km = [1, 5, 10, 25, 50]
stats = {
    "n_exp": exp_xy.shape[0],
    "min_km": float(np.min(dist_km)),
    "median_km": float(np.median(dist_km)),
    "p95_km": float(np.quantile(dist_km, 0.95)),
    "max_km": float(np.max(dist_km)),
    "within_km": {t: int((dist_km <= t).sum()) for t in thr_km},
}
print(stats)

# ---- If many distances are huge, assignments likely miss; check your centr_D mapping ----
if "centr_D" in entity_future.exposures.gdf.columns:
    centr = entity_future.exposures.gdf["centr_D"].to_numpy()
    print({
        "centr_D_nan_frac": float(np.mean(np.isnan(centr))),
        "centr_D_unique": int(len(np.unique(centr[~np.isnan(centr)]))),
    })


EXP bbox (lat_min,lat_max,lon_min,lon_max): (5.762692553363922, 20.45841426613888, 97.4537335977091, 105.6284026831805)
HAZ bbox (lat_min,lat_max,lon_min,lon_max): (5.130434782609598, 21.0, 96.0, 105.99999999999943)
{'n_exp': 453248, 'min_km': 0.03087581878708878, 'median_km': 9.445195984526364, 'p95_km': 14.206454004224458, 'max_km': 16.660523729508746, 'within_km': {1: 2638, 5: 64472, 10: 254083, 25: 453248, 50: 453248}}
{'centr_D_nan_frac': 0.0, 'centr_D_unique': 975}


In [18]:
import numpy as np
import scipy.sparse as sp

# exposures → unique centroid indices used by assets
exp_idx = entity_future.exposures.gdf["centr_D"].astype(int).to_numpy()
uniq_idx = np.unique(exp_idx)

# pick one event row to test (e.g., event 0)
row = hazard_future.intensity.getrow(0)             # CSR, shape (1, n_centroids)
vals = row[:, uniq_idx].toarray().ravel()           # intensities at asset-used centroids

print({
    "n_asset_centroids": uniq_idx.size,
    "nonzero_on_assets": int((vals > 0).sum()),
    "min": float(vals.min()) if vals.size else None,
    "p50": float(np.median(vals)) if vals.size else None,
    "p95": float(np.quantile(vals, 0.95)) if vals.size else None,
    "max": float(vals.max()) if vals.size else None,
})

# optional: treat “near zero” as below a domain threshold
thr = 1e-6
print({
    "near_zero_count": int((vals <= thr).sum()),
    "near_zero_frac": float((vals <= thr).mean()),
})

# overall check across ALL events on asset-used centroids
sub_all = hazard_future.intensity[:, uniq_idx]      # sparse (events x asset-centroids)
max_per_centroid = np.asarray(sub_all.max(axis=0)).ravel()
print({
    "asset_centroids_with_any_signal": int((max_per_centroid > 0).sum()),
    "asset_centroids_all_zero": int((max_per_centroid <= thr).sum()),
})


{'n_asset_centroids': 975, 'nonzero_on_assets': 0, 'min': -3.5, 'p50': -1.6465805768966675, 'p95': -1.3205202341079714, 'max': -1.1166930198669434}
{'near_zero_count': 975, 'near_zero_frac': 1.0}


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all().

In [ ]:
hazard_future.centroids.coord

# array([[ 21.        ,  96.        ],
#        [ 21.        ,  96.2173913 ],
#        [ 21.        ,  96.43478261],
#        ...,
#        [  5.13043478, 105.56521739],
#        [  5.13043478, 105.7826087 ],
#        [  5.13043478, 106.        ]])

array([[ 21.        ,  96.        ],
       [ 21.        ,  96.2173913 ],
       [ 21.        ,  96.43478261],
       ...,
       [  5.13043478, 105.56521739],
       [  5.13043478, 105.7826087 ],
       [  5.13043478, 106.        ]])

In [ ]:
hazard_future.intensity

# <5x3478 sparse matrix of type '<class 'numpy.float32'>'
# 	with 17390 stored elements in Compressed Sparse Row format>

<5x3478 sparse matrix of type '<class 'numpy.float32'>'
	with 17390 stored elements in Compressed Sparse Row format>

In [ ]:
# Example CLIMADA
entity_present = Entity.from_excel(
    DATA_ENTITIES_DIR / "entity_TODAY_THA_D_tree_crops.xlsx"
)
# Set exposure ref year
entity_present.exposures.ref_year = 2024
entity_present.check()

# Set future Entity
entity_future = deepcopy(entity_present)
entity_future.exposures.ref_year = 2050
growth = 0.029
entity_future.exposures.gdf["value"] = entity_future.exposures.gdf.value.values * (1 + growth) ** (
    entity_future.exposures.ref_year - entity_present.exposures.ref_year
)
entity_future.check()

# Set present Hazard
hazard_present = Hazard.from_hdf5(
    DATA_HAZARDS_DIR / "hazard_D_THA_historical.h5",
)
hazard_present.units = "m"
hazard_present.centroids.set_geometry_points()
hazard_present.intensity_thres = -4
hazard_present.check()

# Set future hazard
hazard_future = Hazard.from_hdf5(
    DATA_HAZARDS_DIR / "hazard_HW_EGY_rcp45.h5",
    attrs={
        "frequency": np.array([0.5, 0.2, 0.1, 0.04]),
        "event_id": np.array([1, 2, 3, 4]),
        "units": "number of days",
    },
    haz_type="HW",
    band=[1, 2, 3, 4],
)
hazard_future.units = "number of days"
hazard_future.centroids.set_geometry_points()
hazard_future.intensity_thres = 0
hazard_future.check()

# Calculate impact
impact_present = ImpactCalc(
    entity_present.exposures, entity_present.impact_funcs, hazard_present
).impact()
impact_future = ImpactCalc(
    entity_future.exposures, entity_future.impact_funcs, hazard_future
).impact()

In [ ]:
# List DataTypeInfos
data_type_infos = client.list_data_type_infos()
exposure_data_types = [
    data_type_info.data_type
    for data_type_info in data_type_infos
    if data_type_info.data_type_group == "exposures"
]
hazard_data_types = [
    data_type_info.data_type
    for data_type_info in data_type_infos
    if data_type_info.data_type_group == "hazard"
]

print(
    f"Available Exposure data types in CLIMADA API for all countries:\n{exposure_data_types}")
print(
    f"Available Hazard data types in CLIMADA API for all countries:\n{hazard_data_types}")

In [ ]:
# Available country Exposures and Hazard data types
dataset_infos = client.list_dataset_infos(
    properties={
        "country_name": country_name,
    }
)

exposure_data_types = list(
    set(
        [
            dataset_info.data_type.data_type
            for dataset_info in dataset_infos
            if dataset_info.data_type.data_type_group == "exposures"
        ]
    )
)
hazard_data_types = list(
    set(
        [
            dataset_info.data_type.data_type
            for dataset_info in dataset_infos
            if dataset_info.data_type.data_type_group == "hazard"
        ]
    )
)

print(
    f"Available Exposure data types in CLIMADA API for {country_name}: {exposure_data_types}")
print(
    f"Available Hazard data types in CLIMADA API for {country_name}: {hazard_data_types}")

In [ ]:
# Available climate scenarios and time horizons for specific hazard type in countries Thailand and Egypt
if hazard_type == "river_flood" or "wildfire":
    available_scenarios = list(
        set(
            [
                dataset_info.properties["climate_scenario"]
                for dataset_info in dataset_infos
                if dataset_info.data_type.data_type == hazard_type
            ]
        )
    )
    available_time_horizons = list(
        set(
            [
                dataset_info.properties["year_range"]
                for dataset_info in dataset_infos
                if dataset_info.data_type.data_type == hazard_type
            ]
        )
    )
if hazard_type == "tropical_cyclone":
    available_scenarios = list(
        set(
            [
                dataset_info.properties["climate_scenario"]
                for dataset_info in dataset_infos
                if dataset_info.data_type.data_type == hazard_type
            ]
        )
    )
    available_time_horizons = list(
        set(
            [
                dataset_info.properties.get("ref_year")
                for dataset_info in dataset_infos
                if dataset_info.data_type.data_type == hazard_type
            ]
        )
    )
if hazard_type == "earthquake":
    available_scenarios = []
    available_time_horizons = []
if hazard_type == "flood":
    available_scenarios = []
    available_time_horizons = list(
        set(
            [
                dataset_info.properties["year_range"]
                for dataset_info in dataset_infos
                if dataset_info.data_type.data_type == hazard_type
            ]
        )
    )

print(
    f"Available climate scenarios for hazard type {hazard_type} in country {country_name}: {available_scenarios}"
)
print(
    f"Available time horizons for hazard type {hazard_type} in country {country_name}: {available_time_horizons}"
)

In [ ]:
# Available country Hazard DatasetInfos
hazard_dataset_infos = client.list_dataset_infos(
    properties={
        "data_type": "river_flood",
        "country_name": "Thailand",
        "climate_scenario": "rcp26",
        "year_range": "2030_2050",
    }
)
hazard_dataset_infos

In [ ]:
# Database
import pandas as pd
import sqlite3

class DataFrameSQLite:
    def __init__(self, db_path):
        self.db_path = db_path

    def save_dataframe(self, df, table_name):
        """
        Save a DataFrame to the SQLite database.
        
        :param df: DataFrame to save.
        :param table_name: Name of the table to save the DataFrame to.
        """
        with sqlite3.connect(self.db_path) as conn:
            df.to_sql(table_name, conn, if_exists='replace', index=False)
            print(f"DataFrame saved to {table_name} table.")

    def read_dataframe(self, table_name):
        """
        Read a DataFrame from the SQLite database.
        
        :param table_name: Name of the table to read the DataFrame from.
        :return: DataFrame read from the database.
        """
        with sqlite3.connect(self.db_path) as conn:
            df = pd.read_sql(f"SELECT * FROM {table_name}", conn)
        return df

# Example usage
db_path = 'my_data.db'  # Path to your SQLite database file
df_sqlite = DataFrameSQLite(db_path)

# Assuming you have a DataFrame `df` to save
df_sqlite.save_dataframe(exp_gdf, 'exposure')

# To read the saved DataFrame from the database
df_read = df_sqlite.read_dataframe('exposure')
print(df_read)


In [ ]:
# Database
import pandas as pd
import sqlite3


class ExcelToSQLite:
    def __init__(self, db_path):
        self.db_path = db_path

    def read_excel_to_df(self, excel_path, sheet_name):
        """Reads a specified sheet from an Excel file into a DataFrame."""
        return pd.read_excel(excel_path, sheet_name=sheet_name)

    def adjust_df_columns(self, df, columns_to_keep=None):
        """Adjusts DataFrame columns based on the provided list. If None, keeps all columns."""
        if columns_to_keep is not None:
            df = df[columns_to_keep]
        return df

    def save_df_to_sqlite(self, df, table_name):
        """Saves a DataFrame to an SQLite table, appending data if the table already exists."""
        with sqlite3.connect(self.db_path) as conn:
            df.to_sql(table_name, conn, if_exists="append", index=False)

    def read_table(self, table_name):
        """Reads a table from SQLite database into a DataFrame."""
        with sqlite3.connect(self.db_path) as conn:
            return pd.read_sql(f"SELECT * FROM {table_name}", conn)


# Example usage
db_path = "climadera.db"  # Path to your SQLite database file
excel_path = DATA_ENTITIES_DIR / "3_entity_TODAY_THAI_D_USD_1_modified.xlsx"  # Path to your Excel file
excel_to_sqlite = ExcelToSQLite(db_path)

# Define the sheets and corresponding table names
sheets_tables = {
    "assets": "exposures",
    "impact_functions": "impact_functions",
    "measures": "measures",
    "discount": "discount_rates",
    "names": "names",
}

# Iterate over sheets and tables, read, adjust (if needed), and save to SQLite
for sheet, table in sheets_tables.items():
    df = excel_to_sqlite.read_excel_to_df(excel_path, sheet)

    # Here you can define which columns to keep for each table if needed, e.g.:
    # if table == 'exposures':
    #     columns_to_keep = ['Column1', 'Column2']
    #     df = excel_to_sqlite.adjust_df_columns(df, columns_to_keep)
    # else:
    #     df = excel_to_sqlite.adjust_df_columns(df)

    excel_to_sqlite.save_df_to_sqlite(df, table)

In [ ]:
# Database
import pandas as pd
import sqlite3


class ExcelToSQLite:
    def __init__(self, db_path):
        self.db_path = db_path

    def read_excel_and_save(self, excel_path, tabs_columns_mapping):
        """
        Read specified tabs from an Excel file and save them to SQLite database.

        :param excel_path: Path to the Excel file.
        :param tabs_columns_mapping: Dict mapping tab names to their columns adjustments.
        """
        with sqlite3.connect(self.db_path) as conn:
            for tab, columns in tabs_columns_mapping.items():
                df = pd.read_excel(excel_path, sheet_name=tab)

                # Adjust columns if specified
                if columns:
                    df = df[columns]

                df.to_sql(tab, conn, if_exists="append", index=False)
                print(f"Data from {tab} tab saved to {tab} table.")

    def read_table(self, table_name):
        """
        Read data from a specified table in the SQLite database.

        :param table_name: Name of the table to read data from.
        :return: DataFrame with the table data.
        """
        with sqlite3.connect(self.db_path) as conn:
            return pd.read_sql(f"SELECT * FROM {table_name}", conn)


# Example usage
db_path = "climadera.db"  # Path to your SQLite database file
excel_path = DATA_ENTITIES_DIR / "3_entity_TODAY_THAI_D_USD_1_modified.xlsx"  # Path to your Excel file
tabs_columns_mapping = {
    "assets": None,  # Specify columns as a list if you want to adjust them, or None to include all
    "impact_functions": None,
    "measures": None,
    "discount": None,
    "names": None,
}

excel_to_sqlite = ExcelToSQLite(db_path)
excel_to_sqlite.read_excel_and_save(excel_path, tabs_columns_mapping)

# Reading data from one of the tables
df_assets = excel_to_sqlite.read_table("assets")
print(df_assets)